This notebook aims to study the vulnerabilities and how their fixes relate to backported updates. 

The data we relied on are subject to a non-disclosure agreement. That means we are not allowed to share these data, so you'll have to trust us ;)

In [1]:
import pandas
import numpy as np
import matplotlib
import seaborn

from IPython.display import display

%matplotlib inline

In [2]:
FIG_SIZE = (8, 3)
FIG_SIZE_WIDE = (8, 2.5)

ECOSYSTEMS = ['NPM', 'Rubygems']
DATE_RANGE = pandas.to_datetime('2015-01-01'), pandas.to_datetime('2020-01-01')
CENSOR_DATE = pandas.to_datetime('2020-01-12')

PALETTE = seaborn.color_palette()
PAL_REL = np.take(seaborn.color_palette('muted'), [3, 8, 2, 0], axis=0)
COLORS = {'NPM': PALETTE[1], 'Rubygems': PALETTE[3]}

matplotlib.rcParams['figure.figsize'] = FIG_SIZE
matplotlib.rcParams['legend.framealpha'] = 1
matplotlib.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'

SEED = 12345
SAVEFIG = False

def _savefig(fig, name):
    import os
    fig.savefig(
        os.path.join('..', 'figures', '{}.pdf'.format(name)),
        bbox_inches='tight'
    )
    
savefig = _savefig if SAVEFIG else lambda x, y: None

# Dataset

In [3]:
df_vuln = (
    pandas.read_csv('../data-raw/vulnerabilities.csv.gz', index_col=0, infer_datetime_format=True, parse_dates=['published', 'disclosed'])
    .rename(columns={
        'Id': 'id',
        'vuln_name': 'vulnerability', 
        'base': 'ecosystem', 
        'cvssScore': 'score',
        'fixedIn': 'fix', 
        'affecting': 'affect',
    })
    .replace({'ecosystem': {'npm': 'NPM', 'RubyGems': 'Rubygems'}})
)

In [56]:
df_vuln.describe(datetime_is_numeric=True)

,published,disclosed,score
count,929,929,929.000000
mean,2018-02-09 10:07:37.265877504,2016-09-24 22:53:20.861141248,6.355974
min,2013-06-23 00:00:00,2006-08-14 00:00:00,1.800000
25%,2016-12-17 00:00:00,2015-01-06 00:00:00,5.300000
50%,2018-02-14 00:00:00,2017-05-31 00:00:00,6.500000
75%,2019-06-14 00:00:00,2019-01-08 00:00:00,7.500000
max,2020-04-10 00:00:00,2020-12-13 00:00:00,10.000000
std,NaN,NaN,1.537565


In [160]:
df_vuln.head()

,id,package,published,disclosed,severity,vulnerability,ecosystem,score,fix,affect,required,fixed
159,SNYK-JS-NODERED-541514,node-red,2020-01-12,2020-01-11,medium,Cross-site Scripting (XSS),NPM,4.6,0.20.7,<0.20.7,True,True
163,SNYK-JS-HOTFORMULAPARSER-541328,hot-formula-parser,2020-01-10,2019-12-18,high,Arbitrary Code Injection,NPM,7.6,3.0.1,<3.0.1,True,True
165,SNYK-JS-HAPIWRECK-541185,@hapi/wreck,2020-01-09,2019-01-31,medium,Prototype Poisoning,NPM,5.6,14.1.4,<14.1.4,True,True
167,SNYK-JS-HAPIBOOM-541183,@hapi/boom,2020-01-09,2013-03-30,medium,Cross-site Scripting (XSS),NPM,6.5,0.3.8,<0.3.8,True,True
175,SNYK-JS-NODESASS-540988,node-sass,2020-01-08,2017-08-18,high,Denial of Service (DoS),NPM,7.5,4.4.0,<4.4.0,True,True


## Data selection

Not all vulnerabilities are of interest for our work. 
First, only the ones affecting one of the packages we considered are useful. Second, only the ones that are fixed are interesting to study (indeed, there is no hope to find a backported fix if there is no fix!). 

In [4]:
df_required = dict()
df_dependents = dict()

for ecosystem in ECOSYSTEMS:
    print('Loading', ecosystem)
    print('.. required packages')
    df_required[ecosystem] = (
        pandas.read_csv(
            '../data/{}-required.csv.gz'.format(ecosystem),
            parse_dates=['date'],
            infer_datetime_format=True,
        )
    )
    
    print('.. dependent packages')
    df_dependents[ecosystem] = (
        pandas.read_csv(
            '../data/{}-dependents.csv.gz'.format(ecosystem),
        )
    )
print('Merging...')

df_required = pandas.concat([v.assign(ecosystem=k) for k,v in df_required.items()])
df_dependents = pandas.concat([v.assign(ecosystem=k) for k,v in df_dependents.items()])

print('Done!')

Loading NPM
.. required packages
.. dependent packages
Loading Rubygems
.. required packages
.. dependent packages
Merging...
Done!


In [5]:
df_vuln = (
    df_vuln
    .merge(
        df_required[['ecosystem', 'package']]
        .drop_duplicates()
        .assign(required=True),
        how='left',
        on=['ecosystem', 'package'],
    )
    .fillna({'required': False})
    .assign(fixed=lambda d: ~d['fix'].isin(['undefined']))
)

In [6]:
(
    df_vuln
    .groupby(['ecosystem', 'required', 'fixed'])
    .agg({'vulnerability': 'count'})
)

vulnerability
ecosystem required fixed               
NPM       False    False           1022
                   True             505
          True     False             47
                   True             614
Rubygems  False    False            110
                   True             247
          True     False             14
                   True             315

We'll focus on the ones being "required" and "fixed". 

In [174]:
df_vuln = df_vuln.query('required and fixed')

In [145]:
(
    df_vuln
    .groupby(['ecosystem', 'severity'])
    .agg({'vulnerability': 'count', 'package': 'nunique'})
)

vulnerability  package
ecosystem severity                        
NPM       critical             29       22
          high                198      129
          low                  40       36
          medium              303      183
Rubygems  critical              8        7
          high                 82       36
          low                  15        9
          medium              201       75

In [146]:
df_vuln.sample(n=10, random_state=SEED)

,id,package,published,disclosed,severity,vulnerability,ecosystem,score,fix,affect,required,fixed
2751,SNYK-RUBY-ACTIONPACK-20123,actionpack,2016-09-21,2013-12-02,medium,Cross-site Scripting (XSS),Rubygems,4.3,4.0.2||3.2.16,"< 4.0.2, >= 3.3||< 3.2.16",True,True
2222,SNYK-RUBY-RAILS-536097,rails,2019-11-28,2010-10-28,medium,Improper Input Validation,Rubygems,6.5,2.3.10||3.0.1,">=2.3.9, <2.3.10||>=3.0.0, <3.0.1",True,True
1999,SNYK-JS-REACT-10193,react,2016-11-08,2015-03-17,high,Cross-site Scripting (XSS),NPM,7.1,0.14.0,>=0.0.1 <0.14.0,True,True
2265,SNYK-RUBY-GOOGLEPROTOBUF-472379,google-protobuf,2019-10-09,2017-09-25,high,Integer Overflow,Rubygems,8.8,3.4.0,<3.4.0,True,True
2423,SNYK-RUBY-ACTIVEADMIN-20464,activeadmin,2017-10-17,2010-07-15,medium,Cross-site Scripting (XSS),Rubygems,6.5,0.1.0,<0.1.0,True,True
437,SNYK-JS-VM2-473188,vm2,2019-10-15,2019-04-07,high,Sandbox Bypass,NPM,8.3,3.6.11,<3.6.11,True,True
1753,SNYK-JS-GENERATORJHIPSTER-10453,generator-jhipster,2017-03-06,2015-10-05,medium,Timing Attack,NPM,5.3,2.23.0,>=2.0.1 <2.23.0,True,True
2113,SNYK-JS-HAPI-10074,hapi,2016-01-05,2015-12-28,low,Potentially loose security restrictions,NPM,6.5,11.1.4,<11.1.4,True,True
1007,SNYK-JS-PONSE-12211,ponse,2018-08-20,2018-07-18,high,Directory Traversal,NPM,7.5,2.0.3,<2.0.3,True,True
901,SNYK-JS-STATICEVAL-173693,static-eval,2019-02-14,2019-01-03,high,Arbitrary Code Execution,NPM,7.3,2.0.2,<2.0.2,True,True


## Preprocessing

The dataset contains expressions to capture which versions are affected, and in which versions a vulnerability was fixed. 
We'll parse these expressions to convert them to intervals, so we can manipulate them more easily. 
The notation used is close to the one of Packagist, so we'll use our Packagist parser.

In [161]:
import sys

sys.path.append('../data')

from parsers import parse_or_empty, PackagistParser
from version import Version

parser = PackagistParser()

intervals = dict()

for expr in df_vuln.affect.drop_duplicates():
    intervals[expr] = parse_or_empty(parser, expr)
    
for expr in df_vuln.fix.drop_duplicates():
    intervals[expr] = parse_or_empty(parser, expr)

How many expressions did we successfully convert?

In [162]:
print('expressions:', len(intervals))
print('converted to non-empty:', len([k for k,v in intervals.items() if not v.empty]))
print('proportion:', len([k for k,v in intervals.items() if not v.empty]) / len(intervals))

print()

print('converted to empty:', len([k for k,v in intervals.items() if v.empty]))
print('\n'.join([k for k,v in intervals.items() if v.empty]))

expressions: 1237
converted to non-empty: 1227
proportion: 0.9919159256265158

converted to empty: 10
>=3.0.0-alpha.1 <3.0.0
>=10.0.0-alpha.0 <10.0.0-beta.1
>=3.0.0-rc1 <3.0.0
>=2.0.0-alpha <2.0.0-alpha8
<0.0.0
>=4.0.0-alpha, <4.0.0-beta.2
>=5.2.2, <5.2.2.1
>=4.2.11, <4.2.11.1||>=5.0.7, <5.0.7.2||>=5.1.6, <5.1.6.2||>=5.2.2, <5.2.2.1
< 1.1.rc
1.1.rc


We label each release as being "affected" or not.

In [164]:
df_affected = (
    df_required
    .merge(
        df_vuln[['id', 'ecosystem', 'package', 'affect', 'fix']],
        how='inner',
        on=['ecosystem', 'package'],
    )
    # Ignore those for which we cannot deduce a fixed or affected release
    [lambda d: ~d['affect'].isin([k for k,v in intervals.items() if v.empty])]
    [lambda d: ~d['fix'].isin([k for k,v in intervals.items() if v.empty])]
    # Tag releases as fixed or affected
    .assign(affected=lambda d: 
        d.apply(axis=1, func=lambda s: 
            Version.from_string(s.version) in intervals[s.affect],
        )
    )
    .assign(fixed=lambda d: 
        d.apply(axis=1, func=lambda s: 
            Version.from_string(s.version) in intervals[s.fix],
        )
    )
)

In [165]:
(
    df_affected
    .groupby(['ecosystem'])
    .agg({
        'id': 'nunique', 
        'package': 'nunique',
        'version': 'count',
        'affected': 'sum',
    })
)

,id,package,version,affected
ecosystem,,,,
NPM,565,296,55288,29902
Rubygems,299,93,33673,13807


## Affected major branches

Our focus being on backporting updates in previous major branches, let's annotate, for each major of each package, whether it was affected and fixed.
We remove from our dataset all packages for which we do not have any affected or fixed release. 

In [189]:
df_major = (
    df_affected
    .groupby(['ecosystem', 'id', 'package', 'major'], as_index=False)
    .agg(
        affected=('affected', 'max'), 
        has_fix=('fixed', 'max'), 
        affect=('affect', 'first'),
        minrank=('rank', 'min'),
        maxrank=('rank', 'max'),
    )
    .eval('fixed = affected and has_fix')
    .eval('notfixed = affected and not has_fix')
    
    .groupby(['ecosystem', 'id', 'package'], as_index=False)
    .filter(lambda g: g.has_fix.max() & g.affected.max())
)

In [192]:
(
    df_major
    .groupby(['ecosystem'])
    .agg({
        'id': 'nunique', 
        'package': 'nunique',
        'major': 'count',
        'affected': 'sum',
        'fixed': 'sum',
        'has_fix': 'sum',
        'notfixed': 'sum',
    })
)

,id,package,major,affected,fixed,has_fix,notfixed
ecosystem,,,,,,,
NPM,529,287,2356,1346,524,578,822
Rubygems,290,90,1215,640,326,353,314


We distinguish between the three following situations: 

 - A major is affected by a vulnerability, and is not fixed (affected = True, fixed = False);
 - A major is affected by a vulnerability and has a fix deployed (affected = True, fixed = True);
 - A major is affected by a vulnerability, and a fix is deployed as next major (next major has affected = False, and has_fix = True).
 
So in the above table, the difference between "fixed" and "has_fix" corresponds to the number of vulnerabilities having been fixed in a major release.

The number of backported fixes can be deduced from this dataframe: any vulnerability being fixed in at least two distinct major branches imply at least one backport (the number of major branches being the number of backport + 1). 

In [242]:
(
    df_major
    .groupby(['ecosystem', 'package', 'id'], as_index=False)
    .agg(
        number_of_fixes=('has_fix', 'sum'),
        number_of_fixed=('fixed', 'sum'),
        number_of_affected=('affected', 'sum'),
    )
    .eval("""
    fixed_by_next_major = number_of_fixes == 1 and number_of_fixed == 0
    fixed_in_current_major = number_of_fixes == 1 and number_of_fixed == 1
    fixed_with_backport = number_of_fixes > 1
    """)
    .assign(strategy=lambda d: d[['fixed_by_next_major', 'fixed_in_current_major', 'fixed_with_backport']].idxmax(axis=1))
    .groupby(['ecosystem', 'strategy'])
    .agg(
        vulnerabilities=('id', 'nunique'),
        packages=('package', 'nunique'), 
        cases=('package', 'count'),
    )
    .pipe(lambda df:
        pandas.concat([
            df, 
            df
            .groupby(['ecosystem'])
            .apply(lambda g: g / g.sum())           
            .rename(columns=lambda s: 'prop_{}'.format(s))
        ], axis=1)
    )
    .style
    .format('{:.1%}', subset=['prop_vulnerabilities', 'prop_packages', 'prop_cases'])
    .background_gradient(vmin=0, vmax=1, subset=['prop_vulnerabilities', 'prop_packages', 'prop_cases'])
)


Let's look at the number of fixed/affected branches per package:

In [213]:
(
    df_major
    .groupby(['ecosystem', 'id', 'package'], as_index=False)
    .agg({
        'major': 'count',
        'affected': 'sum', 
        'has_fix': 'sum',
        'fixed': 'sum',
        'notfixed': 'sum',
    })
    .eval("""
    p_affected = affected / major
    p_fixed = fixed / affected
    p_notfixed = notfixed / affected
    """)
    .groupby('ecosystem')
    .describe()
    [['major', 'affected', 'fixed', 'has_fix', 'p_affected', 'p_fixed', 'p_notfixed']]
    .loc[:, (slice(None), ['mean', '50%'])]    
)

major       affected          fixed        has_fix       \
               mean  50%      mean  50%      mean  50%      mean  50%   
ecosystem                                                               
NPM        4.453686  3.0  2.544423  2.0  0.990548  1.0  1.092628  1.0   
Rubygems   4.189655  4.0  2.206897  2.0  1.124138  1.0  1.217241  1.0   

          p_affected             p_fixed      p_notfixed       
                mean       50%      mean  50%       mean  50%  
ecosystem                                                      
NPM         0.717898  0.800000  0.576808  0.5   0.423192  0.5  
Rubygems    0.616872  0.571429  0.636839  0.5   0.363161  0.5

This shows that on average, from 57% to 80% of the major branches are affected by the vulnerability. 
A fix is deployed in 58% to 64% of the affected major branches (leaving the other major branches affected).

In 50% of the cases, a vulnerability affects 2 major branches (out of 3 or 4), and is fixed in 1 major branch, leaving 1 major branch affected.

Let's have a look at the proportion of vulnerabilities in function of the number of affected and fixed major branches.

In [214]:
(
    df_major
    .groupby(['ecosystem', 'id', 'package'], as_index=False)
    .agg({
        'affected': 'sum', 
        'fixed': 'sum',
    })
    .assign(fixed=lambda d: d.fixed.where(d.fixed <= 4, '5+'))
    .assign(affected=lambda d: d.affected.where(d.affected <= 4, '5+'))
    .groupby(['fixed', 'affected'])
    .id
    .count()
    .unstack()
    .pipe(lambda df: df / df.sum().sum())
    .assign(total=lambda d: d.sum(axis=1))
    .style
    .format('{:.2%}')
    .background_gradient(axis=None)
)

affected,1,2,3,4,5+,total
fixed,,,,,,
0,2.93%,1.83%,0.85%,0.37%,1.71%,7.69%
1,35.41%,22.22%,9.16%,7.33%,8.30%,82.42%
2,nan%,2.93%,2.81%,1.71%,1.59%,9.04%
3,nan%,nan%,0.12%,nan%,0.37%,0.49%
4,nan%,nan%,nan%,nan%,0.12%,0.12%
5+,nan%,nan%,nan%,nan%,0.24%,0.24%


Around 7.6% of the vulnerabilities are only fixed in a new major release (i.e., no backport).

Most of the vulnerabilities are fixed in at least one of the affected major branch (82%). This is not surprising when there is only one major branch being affected (35%) but should be considered as unsafe when multiple major are subject to the vulnerability (65%). 

## Affected dependent packages

It would be interesting to see how many dependent packages are still affected by each vulnerability. We expect this number to be lower for packages having backported a security fix. 
However, some vulnerabilities are old(er), and some of them affect (very) old versions, implying that any comparison would be biased. Moreover, some dependent packages could have been abandoned, hence not adopting a newer version (fixed or not) anyway. Finally, we only have the dependencies for the latest snapshot, hence we will simply count the (absolute) number of dependent packages that: 

 - are still relying on an older major branch affected by the vulnerability, and not fixed;
 - are still relying on an older major branch affected by the vulnerability, and fixed (i.e., they benefit from the backported fix).

**TODO**: Change the following analysis to take into account our new (and accurate) way of identifying backported updates!

In [265]:
df_aff_dep = (
    df_major
    # Get latest major
    .merge(
        df_major
        .groupby(['ecosystem', 'package'], as_index=False)
        .agg(latest_major=('major', 'max')),
        how='inner',
        on=['ecosystem', 'package'],
    )
    # Get latest fix to identify backports
    .merge(
        df_major
        .query('has_fix')
        .groupby(['ecosystem', 'package'], as_index=False)
        .agg(latest_fixed_major=('major', 'max')),
        how='inner', 
        on=['ecosystem', 'package'],
    )
    # Merge dependents
    .merge(
        df_dependents[['ecosystem', 'source', 'target', 'selected']],
        how='inner',
        left_on=['ecosystem', 'package'],
        right_on=['ecosystem', 'target'],
    )
    # Keep selected major
    .query('minrank <= selected <= maxrank')
    .eval('backported = fixed and latest_fixed_major > major')
)

### How many packages depend on the vulnerable ones?

In [266]:
(
    df_aff_dep
    .groupby('ecosystem')
    .agg(
        vulnerabilities=('id', 'nunique'),
        required=('package', 'nunique'), 
        source=('source', 'nunique'),
        dependencies=('source', 'count')
    ) 
)

,vulnerabilities,required,source,dependencies
ecosystem,,,,
NPM,529,287,106938,509700
Rubygems,290,90,7156,73938


### How many packages depend on an affected major branch? 

Remember that it is not meaningful to look at these numbers proportionally to the above ones!

In [267]:
(
    df_aff_dep
    .query('affected')
    .groupby('ecosystem')
    .agg(
        vulnerabilities=('id', 'nunique'),
        required=('package', 'nunique'), 
        source=('source', 'nunique'),
        dependencies=('source', 'count')
    ) 
)

,vulnerabilities,required,source,dependencies
ecosystem,,,,
NPM,481,265,93046,382941
Rubygems,219,72,3161,20423


### How many of these packages depend on a previous (affected) major branch? 

Remember that it is not meaningful to look at these numbers proportionally to the above ones!

In [268]:
(
    df_aff_dep
    .query('affected and major < latest_major')
    .groupby('ecosystem')
    .agg(
        vulnerabilities=('id', 'nunique'),
        required=('package', 'nunique'), 
        source=('source', 'nunique'),
        dependencies=('source', 'count')
    ) 
)

,vulnerabilities,required,source,dependencies
ecosystem,,,,
NPM,322,172,11543,26806
Rubygems,147,42,588,2012


### How many dependent packages benefit from a backport?

These numbers can be compared with the above ones. 

In [269]:
(
    df_aff_dep
    .query('affected and fixed and backported')
    .groupby('ecosystem')
    .agg(
        vulnerabilities=('id', 'nunique'),
        required=('package', 'nunique'), 
        source=('source', 'nunique'),
        dependencies=('source', 'count')
    )
)

,vulnerabilities,required,source,dependencies
ecosystem,,,,
NPM,82,34,2092,5829
Rubygems,92,14,112,687


### How many dependent packages would benefit from a backport? 

These numbers can be compared with the ones above the above ones. 

In [270]:
(
    df_aff_dep
    .query('affected and not fixed')
    .groupby('ecosystem')
    .agg(
        vulnerabilities=('id', 'nunique'),
        required=('package', 'nunique'), 
        source=('source', 'nunique'),
        dependencies=('source', 'count')
    )
)

,vulnerabilities,required,source,dependencies
ecosystem,,,,
NPM,201,114,4247,7470
Rubygems,27,16,71,124


Putting everything together: 

In [272]:
(
    df_aff_dep
    .eval('on_latest_major = major == latest_major')
    [['ecosystem', 'id', 'package', 'source', 'backported', 'affected', 'fixed', 'on_latest_major']]
    .groupby(['ecosystem', 'on_latest_major', 'affected', 'fixed', 'backported'])
    .agg(
        vulnerabilities=('id', 'nunique'),
        required=('package', 'nunique'), 
        source=('source', 'nunique'),
        cases=('source', 'count')
    )
)

vulnerabilities  \
ecosystem on_latest_major affected fixed backported                    
NPM       False           False    False False                   164   
                          True     False False                   201   
                                   True  False                   109   
                                         True                     82   
          True            False    False False                   260   
                          True     True  False                   267   
Rubygems  False           False    False False                   130   
                          True     False False                    26   
                                   True  False                    37   
                                         True                     92   
          True            False    False False                   204   
                          True     False False                     1   
                                   True  False                    85   

                                                     required  source   cases  
ecosystem on_latest_major affected fixed backported                            
NPM       False           False    False False             86   13528   35461  
                          True     False False            114    4247    7470  
                                   True  False             83    6964   13507  
                                         True              34    2092    5829  
          True            False    False False            146   40709   91298  
                          True     True  False            159   87936  356135  
Rubygems  False           False    False False             27    1242   11806  
                          True     False False             15      65     117  
                                   True  False             22     483    1208  
                                         True              14     112     687  
          True            False    False False             60    4712   41709  
                          True     False False              1       7       7  
                                   True  False             40    2688   18404